In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

In [2]:
df = pd.read_csv("amazon-food_reviews.csv")

# Apply binary classification to 'Score'
df['Score'] = df['Score'].apply(lambda x: 1 if x >= 3 else 0)
balanced_df = pd.DataFrame()
positive_df = df[df['Score'] == 1].sample(n=5391, random_state=42)  # Downsample positive class
negative_df = df[df['Score'] == 0].sample(n=5391, random_state=42)  # Assume we can take exactly 18391
balanced_df = pd.concat([positive_df, negative_df])


In [3]:
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Preprocess the text
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = word_tokenize(text)  # Tokenize
    text = [word for word in text if word not in stop_words]  # Remove stopwords
    return ' '.join(text)

balanced_df['processed_text'] = balanced_df['Text'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gpava\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
df = balanced_df
df

,Score,Summary,Text,processed_text
0,0,Worst Hot Chocolate I've Ever Had!,I couldn't even taste the chocolate. It tasted...,could n't even taste chocolate . tasted like d...
1,1,Awesome Tea,I drank this tea at least 4 days a week during...,drank tea least 4 days week last 6 weeks pregn...
2,1,WONDERFUL You wont believe!,This brownie mix ROCKS. YOu wont believe how l...,brownie mix rocks . wont believe low cal ! who...
3,0,Just didn't care for it.,I ordered a sample pack that had five of these...,ordered sample pack five . thank goodness . wo...
4,1,"Very good, but a little overboard on the salt",I have a Pasadena popcorn machine from The Gre...,pasadena popcorn machine great northern popcor...
...,...,...,...,...
10777,0,Disappointed: Sad or displeased because someon...,"For years, I used the generic store brand cann...","years , used generic store brand canned tomato..."
10778,1,Love the Taco Bell Sauce!,Happy that I received the Taco Bell Bold & Cre...,happy received taco bell bold & creamy spicy r...
10779,1,Transportable y buen Sonido!,"Muy buen producto. por su tama&ntilde;o, uno l...","muy buen producto . por su tama & ntilde ; , u..."
10780,1,1 year olds LOVE this flavor,This is the one flavor that my 1 year olds wil...,one flavor 1 year olds always eat - matter foo...


In [5]:
X=list(df['processed_text'])

In [6]:
y=list(df['Score'])

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [22]:
from transformers import TFDistilBertForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    warmup_steps=250,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)
    

In [8]:
from transformers import DistilBertTokenizerFast, TFDistilBertForSequenceClassification
import tensorflow as tf

# Load tokenizer and model
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(set(y_train)))

# Tokenize data
train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), y_train)).shuffle(len(y_train)).batch(8)
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), y_test)).batch(16)

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = [
    tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# Train the model
model_history = model.fit(train_dataset, validation_data=test_dataset, epochs=2)

# Evaluate the model
results = model.evaluate(test_dataset)
print("Test loss, Test acc:", results)

# Predict using the model
predictions = model.predict(test_dataset)
print("Predictions:", predictions['logits'][:5])  # Display first 5 predictions



Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Epoch 1/2
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert



1079/1079 [==============================] - 6920s 6s/step - loss: 0.4073 - accuracy: 0.8172 - val_loss: 0.3057 - val_accuracy: 0.8809
Epoch 2/2
135/135 [==============================] - 512s 4s/step - loss: 0.3078 - accuracy: 0.8734
Test loss, Test acc: [0.30775538086891174, 0.8734353184700012]
135/135 [==============================] - 515s 4s/step
Predictions: [[-2.3106117   2.363161  ]
 [ 2.2845702  -2.1746745 ]
 [ 0.64116704 -0.4973068 ]
 [-1.8871561   1.9206743 ]
 [-2.6253      2.6676183 ]]


In [9]:
model.save('Distil_bert-finetune')  


INFO:tensorflow:Assets written to: Distil_bert-finetune\assets


INFO:tensorflow:Assets written to: Distil_bert-finetune\assets


In [ ]:
model.save('distil-bert-tf', save_format='tf')
